In [60]:
import re
import os
import numpy as np
import math
import pickle
import matplotlib.pyplot as plt
from nltk import FreqDist
from nltk.util import ngrams 
from zipfile import ZipFile
from kneebow.rotor import Rotor
from kneed import KneeLocator

Load 2 million word corpus

In [61]:
corpus = []

archive = ZipFile('corpus2mw.zip', 'r')

fileList = archive.namelist()
for file in fileList:
    corpus.append((archive.read(file)).decode('UTF-8'))


Process text (separate special characters from words)

In [63]:
regexp = re.compile('[\w \-]') 

def processText(corpus):
    corp = []
    for text in corpus:
        listT = list(text)
        i = 0
        for  c in listT:
            if (not regexp.search(c) and not listT[i-1]==' ') or not regexp.search(listT[i-1]) and regexp.search(c):
                listT.insert(i, ' ')
            i +=1
        corp.append(''.join(listT))
    return corp

readPickle = False
if readPickle:
    with open('corpusList', 'rb') as fp:
        corpus = pickle.load(fp)
else: 
    corpus =  processText(corpus)

    with open('corpusList', 'wb') as fp:
            pickle.dump(corpus, fp)

Compute ngrams

In [ ]:
def compute_freq_doc(text, minG, maxG):
   freq_dist = FreqDist()
   if len(text) > 1:
       tokens = text.strip().split()
       for i in range(minG, maxG+1):
           grams = ngrams(tokens, i)
           freq_dist.update(grams)

   return dict(freq_dist)


def compute_freq_corpus(minG, maxG):
   freq_dist = FreqDist()
   for text in corpus:
        if len(text) > 1:
            tokens = text.strip().split()
            for i in range(minG, maxG+1):
                grams = ngrams(tokens, i)
                freq_dist.update(grams)

   return dict(freq_dist)


freq_dict = compute_freq_corpus(1, 8)
filtered_dict = {' '.join(key):val for key, val in freq_dict.items() if val > 1}

LocalMax

In [ ]:
def dice(freq, pref_freqs, suff_freqs):
    pref_freqs = list(pref_freqs)
    suff_freqs = list(suff_freqs)
    return 2 * freq / (sum(pref_freqs) / len(pref_freqs) + sum(suff_freqs) / len(suff_freqs))


def scp(freq, pref_freqs, suff_freqs):
    multiplied_freqs = [pref_freq * suff_freq for pref_freq, suff_freq in zip(pref_freqs, suff_freqs)]
    return freq ** 2 * len(multiplied_freqs) / sum(multiplied_freqs)

In [ ]:
relev_expressions={ }


for key, value in filtered_dict.items():
    n= len(key.split(' '))
    if n>3:
        own_freq = value/(len(corpus)-n)
        #print(key)
        ownpref=''
        ownsuf=''
        for i in range(0,n):
            if i==0:
                ownpref += key.split(' ')[i]
            elif i==n-1:
                ownsuf += key.split(' ')[i]
            else:
                ownpref += ' ' + key.split(' ')[i]
                ownsuf += key.split(' ')[i] + ' '
        #print(ownpref)
        #print(ownsuf)
        own_pref_freq= filtered_dict[ownpref]/(len(corpus)-(n-1))
        own_suff_freq= filtered_dict[ownsuf]/(len(corpus)-(n-1))
        gluew= own_freq**2 /(own_pref_freq * own_suff_freq)
        #print(gluew)

        if own_pref_freq>own_suff_freq:
            aux= ownpref
            auxfreqy= own_pref_freq
        else:
            aux= ownsuf
            auxfreqy= own_suff_freq
            
        prefs=''
        sufs=''
        for i in aux.split(' ')[:-1]:
            prefs+=i+' '
        for i in aux.split(' ')[1:]:
            sufs+=i+' '
        
        own_pref_freq= filtered_dict[prefs.strip()]/(len(corpus)-(n-2))
        own_suff_freq= filtered_dict[sufs.strip()]/(len(corpus)-(n-2)) 
       
        gluex= auxfreqy**2 /(own_pref_freq * own_suff_freq)
        #print('x', gluex)
        
        besty=''
        bestfreqy=0
        for cha in filtered_dict.keys():
            if key.split(' ')[:]==(cha.split(' ')[:-1] or cha.split(' ')[1:]):
                #print(key, cha)
                wy= filtered_dict[cha]/(len(corpus)-(n+1))
                if wy>bestfreqy:
                    bestfreqy=wy
                    besty=cha
                          
        if(bestfreqy>0):
            pref=''
            suf=''
            for i in besty.split(' ')[:-1]:
                pref+=i+' '
            for i in besty.split(' ')[1:]:
                suf+=i+' '  
                        
            own_pref_freq= filtered_dict[pref.strip()]/(len(corpus)-(n-2))
            own_suff_freq= filtered_dict[suf.strip()]/(len(corpus)-(n-2)) 
        
            gluey= bestfreqy**2 /(own_pref_freq * own_suff_freq)
            
            re= (gluex+gluey)/2
            #print('re', re)
            #print('gluew', gluew)
            if re<gluew:
                relev_expressions[key]=re
                print(key)
        else:
            continue
    
        
        
        
    

Stop Words

In [ ]:

#unigrams
single_freq_dict = {key:val for key, val in filtered_dict.items() if len(key.split()) == 1}
#sort unigrams by value
single_freq_dict = {k: v for k, v in sorted(single_freq_dict.items(), key=lambda item: item[1])}


values = np.fromiter(single_freq_dict.values(), dtype=float)
stop_words_list = np.stack((np.arange(0, len(single_freq_dict)), values), axis = -1)
list_of_counts = list(single_freq_dict.items())


rotor = Rotor()
rotor.fit_rotate(stop_words_list)
elbow_idx = rotor.get_elbow_index()
print(elbow_idx)  
#rotor.plot_elbow()

kn = KneeLocator(stop_words_list[:,0] ,stop_words_list[:,1], curve='convex', direction='increasing')
print(int(kn.knee))

stop = 0
deltaX = 200
for idx, i, j in zip(range(0, len(values)), values, values[deltaX:]):
    if((j-i)>stop): stop = idx
print(stop)


fig = plt.figure()
ax = plt.gca()
ax.scatter(stop_words_list[:,0] ,stop_words_list[:,1] , s=1,c='blue', marker='.')
ax.set_yscale('log')
#ax.set_xscale('log')
fig.set_size_inches(10, 7)
plt.axvline(x=elbow_idx, color='r', linestyle='--')


relev_exp_unigrams = list_of_counts[:elbow_idx] #not really necessary
stop_words_unigrams = list_of_counts[elbow_idx:]

#TODO filter relevant expression dictionary if w1 or wn in stop_words_unigrams -> delete

Explicit and Implicit Keywords

In [ ]:
def count_RE_in_doc(RE):
    count = 0
    for text in corpus:
        if RE in text:
            count += 1
    
    return count

def freq(RE,doc):
    freq_dict = compute_freq_doc(doc, len(RE.split()), len(RE.split()))
    
    return freq_dict[RE]


def tf_idf(RE, doc_idx):
    doc = corpus(idx)
    
    freq_RE = freq(RE,doc)
    
    return (freq_RE/len(doc))*math.log(len(corpus)/count_RE_in_doc(RE))

def calc_prob(word):
    sum_p = 0
    for doc in corpus:
        sum_p += freq(word, doc)/len(doc.split())
    return (1/len(corpus))*sum_p

def calc_cov(A,B):
    probA = calc_prob(A)
    probB = calc_prob(B)
    sum_p = 0
    for doc in corpus:
        sum_p += (freq(A, doc)/len(doc.split())-probA)*(freq(B, doc)/len(doc.split())-probB)
    return (1/len(corpus)-1)*sum_p

def correlation(A,B):
    return calc_cov(A, B)/(math.sqrt(calc_cov(A, A))*(math.sqrt(calc_cov(B, B))))


  
def IP(A,B):
    count = 0
    sum_dist = 0 
    for doc in corpus:
        if A in doc and B in doc:
            count += 1
            
        
        #TODO sum_dist += calculate closest divided by farthest distances between A and B
        
    
    return 1-(1/count)*sum_dist
        
       
def sem_prox(A,B):
    return correlation(A, B)*math.sqrt(IP(A,B))

def score_implicit(RE,doc):
    return #TODO